# Telecom Churn Prediction

* [Dataset exploration](#Dataset-exploration)
* [Baseline models for churn classification](#Baseline-models-for-churn-classification)
* [Explainig models](#Explainig-models)
* [Uplift models](#Uplift-models)
* [Take out](#Take-out)

### Importing libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pingouin as pg

In [ ]:
data = pd.read_csv('data/telecom_churn.csv')

## Dataset exploration

### EDA

### Relationships

## Baseline models for churn classification

### Baseline models

### Tsne models

## Explainig models

## Uplift models

## Take out